In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('window-size=1920x1480')
options.add_argument('--disable-gpu')  # Last I checked this was necessary.
CHROMEDRIVER_PATH = 'E:/GOMMT/Solve for Tomorrow (6292020)/Selenium/chromedriver.exe'
driver = webdriver.Chrome(CHROMEDRIVER_PATH, options=options)

# DATA

In [4]:
Airlines =  []
DepPlace =  []
DepTime  =  []
ArrPlace =  []
ArrTime  =  []
PriBefore=  []
CouCodes =  []
Message  =  []
PriAfter =  []
driver.get('https://flight.easemytrip.com/FlightList/Index?srch=DEL-Delhi-India|BOM-Mumbai-India|09/07/2020&px=1-0-0&cbn=0&ar=undefined&isow=true&isdm=true&lng=&')
driver.maximize_window()
time.sleep(5)
Bookbutton = driver.find_elements_by_class_name('book-bt-n')
noofflights = len(Bookbutton)
for bt_cnt in range(noofflights):
    Bookbutton = driver.find_elements_by_class_name('book-bt-n')
    webdriver.ActionChains(driver).move_to_element(Bookbutton[bt_cnt]).perform()
    webdriver.ActionChains(driver).click(Bookbutton[bt_cnt]).perform()
    time.sleep(2)
    ### ON BOOKING PAGE
    # FLIGHT DETAILS
    content = driver.page_source
    soup = BeautifulSoup(content)
    Flight = soup.find('div', attrs={'class':'bor pdb10'})
    detail = Flight.find('div',attrs={'class':'fli-cm'}).text
    DeparturePlace = detail.split()[0]
    DepartureTime  =detail.split()[1]
    # print(DeparturePlace)
    # print(DepartureTime)
    detail = Flight.find('div',attrs={'class':'fli-cm1'}).text
    ArrivalPlace = detail.split()[0]
    ArrivalTime  = detail.split()[1]
    # print(ArrivalPlace)
    # print(ArrivalTime)
    Airline = soup.find('div', attrs={'class':'fli1-m-r'}).find('span').text
    # print(Airline)

    # Price Without coupon
    total = soup.find('div', attrs={'id':'divFareSummary'})
    BeforeGrandTotal = total.find('div', attrs={'class':'pr-r1_Rs'}).text.strip()
    # print(BeforeGrandTotal)

    couponscodes = soup.findAll('div', attrs={'class':'coupn_ttl'})
    # print(couponscodes[0].text)
    
    ## SELECTING ALL COUPONS FROM A BOOKING
    checkmark = driver.find_elements_by_class_name('coupn_checkmark')
    i = 0
    for mark in checkmark:
        try :
            webdriver.ActionChains(driver).move_to_element(mark).perform()
            webdriver.ActionChains(driver).click(mark).perform()
        except Exception:
            continue
        time.sleep(2)
        content = driver.page_source
        soup = BeautifulSoup(content)
        msg = soup.find('p', attrs={'id':'easeFareDetails1_promodiv'}).text
        #     print(msg)
        total = soup.find('div', attrs={'id':'divFareSummary'})
        AfterGrandTotal = total.find('div', attrs={'class':'pr-r1_Rs'}).text.strip()
        #     print(AfterGrandTotal)
        Airlines.append(Airline)
        DepPlace.append(DeparturePlace)
        DepTime.append(DepartureTime)
        ArrPlace.append(ArrivalPlace)
        ArrTime.append(ArrivalTime)
        PriBefore.append(BeforeGrandTotal)
        CouCodes.append(couponscodes[i].text)
        Message.append(msg)
        PriAfter.append(AfterGrandTotal)
        i+=1
    driver.execute_script("window.history.go(-1)")
    time.sleep(5)

IndexError: list index out of range

In [5]:
Airlines,CouCodes

(['AirAsia',
  'AirAsia',
  'AirAsia',
  'AirAsia',
  'SpiceJet',
  'SpiceJet',
  'SpiceJet',
  'SpiceJet',
  'Indigo',
  'Indigo'],
 ['EMTNCF',
  'FREEFLIGHT',
  'EMTNCF',
  'FREEFLIGHT',
  'EMTNCF',
  'FREEFLIGHT',
  'EMTNCF',
  'FREEFLIGHT',
  'EMTNCF',
  'FREEFLIGHT'])

In [6]:
print(len(Airlines))
print(len(DepPlace))
print(len(DepTime))
print(len(ArrPlace))
print(len(ArrTime))
print(len(PriBefore))
print(len(PriAfter))
print(len(CouCodes))
print(len(Message))

10
10
10
10
10
10
10
10
10


In [7]:
df = pd.DataFrame({'Airlines': Airlines,
                   'Departure Place': DepPlace,
                   'Departure Time': DepTime,
                   'Arrival Place': ArrPlace,
                   'Arrival Time': ArrTime,
                   'Initial Price': PriBefore,
                   'After Price': PriAfter,
                   'Coupon Code': CouCodes,
                   'Message': Message,}) 
df.to_csv('CoupounsData.csv', index=False, encoding='utf-8')